[View in Colaboratory](https://colab.research.google.com/github/tcglarry/aoi/blob/master/various_training.ipynb)

In [1]:
# Install a Drive FUSE wrapper.
# https://github.com/astrada/google-drive-ocamlfuse
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

gpg: keybox '/tmp/tmpma9g0pad/pubring.gpg' created
gpg: /tmp/tmpma9g0pad/trustdb.gpg: trustdb created
gpg: key AD5F235DF639B041: public key "Launchpad PPA for Alessandro Strada" imported
gpg: Total number processed: 1
gpg:               imported: 1


In [0]:
# Generate auth tokens for Colab
from google.colab import auth
auth.authenticate_user()

In [0]:
# Generate creds for the Drive FUSE library.
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [4]:
!mkdir -p drive
!google-drive-ocamlfuse drive
!ls drive

fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option
aoi_aoi_test1.ipynb  Sample upload.txt
aoi_test	     TAHU PowerPoint Template.pdf
Colab Notebooks      TAHU PowerPoint Template - 柱狀圖 1.ods
PiXNet		     test for excel


In [0]:
!cp drive/aoi_test/train.csv .

In [2]:
from keras.applications.inception_v3 import InceptionV3
from keras.applications.densenet import DenseNet121
from keras.preprocessing import image

from keras.layers import Dense, GlobalAveragePooling2D


import cv2
import numpy as np
import matplotlib.pyplot as plt
import keras.backend as K
import tensorflow as tf
import subprocess
import os
import pickle
from keras.layers import Input, Conv2D, MaxPooling2D, merge, Lambda,UpSampling2D, concatenate, Reshape, Dropout,Cropping2D
from keras.models import Model, load_model
import pandas as pd
import sklearn 
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint,EarlyStopping
from keras.callbacks import Callback
from keras.applications.mobilenet import MobileNet

Using TensorFlow backend.


In [0]:
model = 'inceptionresnet.npy'
src = 'drive/aoi_test/'
#src=''

In [4]:
train_data = np.load(src+model)
print (train_data.shape)
#train_data =  train_data[:,:,:,np.newaxis]
print (train_data.shape)

(2528, 14, 14, 1536)
(2528, 14, 14, 1536)


In [5]:
print ('here')

here


In [7]:
df_y = pd.read_csv('train.csv')
print (df_y['Label'].value_counts())
train_y =  df_y['Label'].values

onehot_encoder = OneHotEncoder(sparse=False)
train_y = train_y.reshape(len(train_y), 1)
print (train_y.shape)
train_label = onehot_encoder.fit_transform(train_y)


print ('beofre', train_label.shape)
#print (df_y['Label'][:10])
#print (train_label[:10,:])

train_label = train_label[:,np.newaxis,np.newaxis,:]

print ('after', train_label.shape)


0    674
5    644
1    492
3    378
4    240
2    100
Name: Label, dtype: int64
(2528, 1)
beofre (2528, 6)
after (2528, 1, 1, 6)


In [8]:
size = (14,14,1536)

inputs = Input(shape=(size))
x = Conv2D(64,(1,1), activation='relu', padding='same')(inputs)
x = Conv2D(64,(3,3), activation='relu', padding='valid')(x)
x = MaxPooling2D(pool_size=(2,2))(x)
x = Conv2D(128,(3,3), activation='relu', padding='valid')(x)
print (x.get_shape())
x = MaxPooling2D(pool_size=(2,2))(x)
print (x.get_shape())

x = Conv2D(128,(2,2), activation='relu', padding='valid')(x)

x = Dense (64, activation='relu')(x)
x = Dropout(0.3)(x)
predictions = Dense (6,activation='softmax')(x)
print ('final',x.get_shape())

(?, 4, 4, 128)
(?, 2, 2, 128)
final (?, 1, 1, 64)


In [9]:
# this is the model we will train
model = Model(inputs=inputs, outputs=predictions)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 14, 14, 1536)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 64)        98368     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 4, 4, 128)         73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 2, 2, 128)         0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 1, 1, 128)         65664     
__________

In [0]:
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='Nadam', loss='categorical_crossentropy',metrics=['accuracy'])

#output_layers = ['predictions']
#model.metrics_tensors += output_layers


In [0]:
# train the model on the new data for a few epochs
use_model = False

if use_model:
    
    if os.path.isfile(src+'inceptionresnet_various_1.h5'):

      model = load_model(src+'inceptionresnet_various_1.h5')
      print ('model laoded')
    else:
      print ('model not exist')


In [12]:
checkpoint = ModelCheckpoint(src+'inceptionresnet_various_1.h5')
earlystop = EarlyStopping(monitor='val_loss',  patience=20,  mode='auto')
#callback_list = [cbk, checkpoint]
callback_list = [checkpoint,earlystop]
for i in range (50):


  
  train_x,val_x,train_y,val_y = train_test_split(train_data,train_label,test_size=0.2, shuffle=True)
  
  model.fit(x=train_x, y=train_y, batch_size=32, epochs=50, verbose=1, callbacks=callback_list, validation_split=0.2, shuffle=True)
  #print ('epoch', i, 'shape=', len(cbk.outputs))


  model.save(src+'inceptionresnet_various_30.h5')
  print ('epoch', i, 'model saved')







Train on 1617 samples, validate on 405 samples
Epoch 1/50
1617/1617 [==============================] - 6s 4ms/step - loss: 1.7158 - acc: 0.2468 - val_loss: 1.6999 - val_acc: 0.2370
Epoch 2/50
1617/1617 [==============================] - 2s 1ms/step - loss: 1.6843 - acc: 0.2449 - val_loss: 1.6907 - val_acc: 0.2469
Epoch 3/50
1617/1617 [==============================] - 2s 1ms/step - loss: 1.6691 - acc: 0.2733 - val_loss: 1.6877 - val_acc: 0.2617
Epoch 4/50
1617/1617 [==============================] - 2s 1ms/step - loss: 1.6594 - acc: 0.2517 - val_loss: 1.6899 - val_acc: 0.2444
Epoch 5/50
1617/1617 [==============================] - 2s 1ms/step - loss: 1.6578 - acc: 0.2684 - val_loss: 1.7044 - val_acc: 0.2494
Epoch 6/50
1617/1617 [==============================] - 2s 1ms/step - loss: 1.6534 - acc: 0.2678 - val_loss: 1.6922 - val_acc: 0.2543
Epoch 7/50
1617/1617 [==============================] - 2s 1ms/step - loss: 1.6387 - acc: 0.2777 - val_loss: 1.7194 - val_acc: 0.2420
Epoch 8/50
1617

KeyboardInterrupt: ignored

In [14]:
model.evaluate(val_x,val_y)

506/506 [==============================] - 1s 1ms/step


[0.5859008725923983, 0.8636363638719551]